In [ ]:
import glob
import os
from platform import system as OS
import numpy as np
import datetime
import matplotlib.pyplot as plt
from IPython.display import clear_output, display, HTML
from pprint import pprint
import warnings
warnings.filterwarnings("ignore")

if "__file__" not in dir():
    %run Animal_Tags.ipynb
    
    
    if OS()=='Linux':
        root="/data"

    elif OS()=="Windows":
        root="C:\\data\\"

    else:
        root="/Users/davidrobbe/Documents/Data/"
    
    print('os:',OS(),'\nroot:',root,'\nImport successful!')
    
def read_entranceTime(fullPath):
        maxTrialDuration=read_in_file(fullPath,"maximum trial duration",extension=".behav_param",valueType=float)
        entranceTimeStr=list(read_in_file(fullPath,"time",extension=".entrancetimes",valueType=str))
        return entranceTimeStr

    #-----------------------------------------------------------------------------------------------   
def read_in_file(fullPath,paramName,extension=".behav_param",exclude=None,valueType=str):
        '''
        Use to read from .behav_param or .entrancetimes
        Look for lines containing "paramName" and not containing "exclude"
        Split them by white spaces 
        example: "treadmill speed:     30.00" becomes ["treadmill","speed:","30.00"])
        Return a list of their last element, in the specified valueType (in example: "30.00")
        '''
        behav=fullPath+extension
        if not os.path.exists(behav):
            #print("No file %s"%behav)
            return []
        result=[]
        with open(behav,"r") as f:
            for line in f:
                if paramName in line:
                    if (exclude is not None) and (exclude in line):
                        continue
                    res=line.split()[-1]
                    #integer or float: replace comma by dots
                    if valueType in [int,float]:
                        res=res.replace(",",".")                 
                    #integer: convert first to float ("0.00" -> 0.00 -> 0)
                    if valueType is int:
                        res=int(float(res))
                    #boolean "TRUE" "FALSE"
                    elif valueType is bool:
                        res=(res.lower()=="true")
                    else:
                        res=valueType(res)
                    result.append(res)
        result=np.asarray(result)
        return result    

def get_current_animals(days_passed=4,thisRoot="/NAS02"):
    now=datetime.date.today()
    thisRoot="/NAS02"
    all_animals=[os.path.basename(path) for path in sorted(glob.glob(os.path.join(thisRoot,"Rat???")))]
    if all_animals==[]:
        print('NAS02 not mounted!')
        return []
    
    last_modifTimes={}
    animalList=[]
    for animal in all_animals:
        experimentsPath=os.path.join(thisRoot,animal,"Experiments")
        if not os.path.exists(experimentsPath):
            continue
        sessionList=[os.path.basename(expPath) for expPath in glob.glob(os.path.join(thisRoot,animal,"Experiments","Rat???_20??_*"))]
        if not sessionList:
            continue
        sessionList=sorted(sessionList)
        lastSessionDate= datetime.datetime.strptime(sessionList[-1][7:17],'%Y_%m_%d').date()
        if (now-lastSessionDate).days<=days_passed:
            animalList.append(animal)
    
    return animalList

## Get animalList from recently updated animals on NAS

In [ ]:
if "__file__" not in dir():
    animalList=get_current_animals(days_passed=5)
    
    print("Number of Animals: %s" %len(animalList))
    print(animalList)


In [ ]:
animalList=['Rat174']
print(animalList)

# Rsync from NAS02

In [ ]:
if "__file__" not in dir():
#     animalList=["Rat174"]
    PathToSourceDataFolder="/NAS02/"
    PathToDestinationDataFolder=root
    RsyncArgument="rsync -auvrR --max-size=50M "
    IncludeArgument="--include '*/' --include '*.tag' --include '*/*.position' --include '*/*.mat' --include '*/*.entrancetimes' --include '*/*.lickbreaktime' --include '*/*.behav_param' --exclude '*/*.coder' --exclude '*/*.seq' --exclude '*/*.avi' "  
    CWD=os.getcwd()                                    
    os.chdir(PathToSourceDataFolder)
    for animal in sorted(animalList):
        print(animal+" in progress...")
        os.system(RsyncArgument +  IncludeArgument + animal + " " + PathToDestinationDataFolder)
        #sys.stdout.flush()
    clear_output()             
    os.chdir(CWD)
    print("Done:",animalList)

# Rsync from NAS02 for EPHY rats

In [ ]:
if "__file__" not in dir():
    animalList=["Rat"]
    PathToSourceDataFolder="/NAS02/"
    PathToDestinationDataFolder=root
    RsyncArgument="rsync -auvrR "
    IncludeArgument="--include '*/' --include '*.tag' --include '*/*.position' --include '*/*.mat' --include '*/*.entrancetimes' --include '*/*.lickbreaktime' --include '*/*.behav_param' --include '*/*.kwd' --exclude '*/*.coder' --exclude '*/*.seq' --exclude '*/*.avi' "  
    CWD=os.getcwd()
    os.chdir(PathToSourceDataFolder)
    for animal in sorted(animalList):
        print("in progress: "+ animal)
        os.system(RsyncArgument +  IncludeArgument + animal + " " + PathToDestinationDataFolder)
        #sys.stdout.flush()
    clear_output()
    os.chdir(CWD)
    print("Done:",animalList)

## return the last 10 median entrance time per animal

In [ ]:
animalList=['Rat181']
for animal in animalList:
    #Get the list of all session
    sessionList=[os.path.basename(expPath) for expPath in glob.glob(os.path.join(root,animal,"Experiments","Rat*"))]
    sessionList=sorted(sessionList)
    LastSessionNames=sessionList[9:]
    meanSessionEntranceTime=[]
    print("**********")
    for i,LastSessionName in enumerate(LastSessionNames):
        
        fullPath=os.path.join(root,animal,"Experiments",LastSessionName,LastSessionName)

        listeEntrance=read_entranceTime(fullPath)
        if len(listeEntrance)==0:
            continue
        
        for i in range(len(listeEntrance)):
            if listeEntrance[i]=="timeout":
                listeEntrance[i]=np.nan
            else:
                listeEntrance[i]=float(listeEntrance[i])
        meanSessionEntranceTime.append(np.nanmedian(listeEntrance))
    print("Animal %s"%animal," sesssion",LastSessionName)
    print(meanSessionEntranceTime[-1])
    print("max entrance time: ",np.nanmax(meanSessionEntranceTime))
    plt.plot(meanSessionEntranceTime,'ro-'),plt.show()

# Load Excel file of Ephy Animals

In [ ]:
class ReadExcelFile:
    def __init__(self,root,animal,fileName=None):
        self.animal=animal
        self.excelPath=''
        if fileName is None:
            path=os.path.join(root,animal,animal)+'*.xls*'
            excelfiles=glob.glob(path)
            assert len(excelfiles)!=0, "No Excel files"+path
            assert len(excelfiles) ==1, "Too many Excel files"+str(excelfiles)
            self.excelPath=excelfiles[0]
        else:
            self.excelPath=fileName
            assert os.path.exist(self.excelPath), "Bad Excel file path"
        
        self.read_excel_file()
    
    def __repr__(self):
        return " ".join(['Excel file at:',self.excelPath])

    
    def read_excel_file(self):
        with pd.ExcelFile(self.excelPath) as file:
            sheets=file.sheet_names
            self.excelData={sheet:pd.read_excel(file,sheet) for sheet in sheets}

class EphyAnimalExcel(ReadExcelFile):
    def __init__(self,*args,**kwargs):
        super().__init__(*args,**kwargs)
        self.check_structure()
        self.tread=self.excelData[self.animal+'settings'].axes[1][1]
    
    def __repr__(self):
        print('current position:')
        pprint(self.current())
        return " ".join(['Excel file for Ephy animal at:',self.excelPath,
                         '\nLast date: {}'.format(self.last_date())])

    def __getitem__(self,args):
        return self.excelData[self.animal].iat[args]
    
    def check_structure(self):
        assert self.excelData[self.animal].axes[1][0]=='tetrodeNumber'
        assert self.excelData[self.animal].axes[1][1]=='initialPosition'
        assert self.excelData[self.animal+'settings'].axes[1][1], 'screw tread was not found'
    
    def movements(self):    
        mov={self.str2date(self.excelData[self.animal].axes[1][i]):
             list(self.excelData[self.animal] [self.excelData[self.animal].axes[1][i]])
             for i in range(2,self.excelData[self.animal].shape[1])}
        return mov
    
    def current(self):
        pos=self.movements().values()
        pos=np.array(list(pos))
        pos=(np.sum(pos,axis=0)/360)*(self.tread/1000)+self.excelData[self.animal]['initialPosition']
        curr= {self.excelData[self.animal].iat[i,0]:pos[i]
               for i in range(self.excelData[self.animal].shape[0])}
        return curr
    
    def position_at(self,date):
        """
        find the position of the tetrodes at a certain date in the past
        """
        if not isinstance(date,datetime.datetime):
            date=self.str2date(date)
        assert isinstance(date,datetime.datetime), "Wrong date format"
        
        mov=self.movements()
        pos=[]
        for dat in mov:
            if dat > date:
                continue
            pos.append(mov[dat])
        pos=np.array(pos)
        pos=(np.sum(pos,axis=0)/360)*(self.tread/1000)+self.excelData[self.animal]['initialPosition']
        
        return pos
    
    def last_date(self):
        return self.str2date(self.excelData[self.animal].axes[1][-1])

    @staticmethod
    def str2date(s,pattern="%Y_%m_%d"):
        return datetime.datetime.strptime(s,pattern)

In [ ]:
EphyAnimalExcel(root,'Rat177')